# NEXUS tool: case study for the NWSAS basin
In this notebook a case study for the NWSAS basin is covered, using the `nexus_tool` package. The water requirements for agricultural irrigation are calculated, then the energy requirements for pumping and desalination of brackish water are estimated and then least-cost options to supply such energy are identified between selected technologies.

First import the package by running the following block:

In [2]:
import nexus_tool

After importing all required packages, the input GIS data is loaded into the variable `df`. Change the `file_path` variable to reflect the name and relative location of your data file.

In [3]:
file_path = r'test_data/NWSAS_input_data.csv'
df = nexus_tool.read_csv(file_path)

## 1. Calculating irrigation water demand
To be able to calculate the water demand for agricultural irrigation, it is required to define crop irrigation calendars for each crop type to be assessed. Then an excel file containing the information of the crop calendars is needed. Such file should look something like this:

|crop|init_start|init_end|dev_start|dev_end|mid_start|mid_end|late_start|late_end|
|:---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|dates|01/11|30/03|31/03|04/05|05/05|30/09|01/10|31/10|
|vegetables|01/11|25/11|26/11|31/12|01/01|07/02|08/02|28/02|
|olives|01/03|30/03|31/06|30/06|01/07|31/08|01/09|30/11|

Change the `file_path` variable to reflect the name and relative location of your data file.


In [4]:
file_path = r'test_data/NWSAS_crop_calendar.xlsx'
crop_calendar = nexus_tool.read_excel(file_path)

### 1.1. Creating the model
Once all input data is loaded. To create a model simply create an instance of the `nexus_tool.Model()` class and store it in a variable name. The `nexus_tool.Model()` class requires a dataframe as input data and another dataframe as crop calendar data. Several other properties and parameter values can be defined by explicitly passing values to them. To see a full list of parameters and their explaination refer to the documentation of the package.

In [5]:
nwsas = nexus_tool.Model(df, crop_calendar = crop_calendar,
                         pumping_hours_per_day=10, deff= 1, aeff= 0.65)

After creating the model you can see the default values of the properties by running `nwsas.print_properties()`. Moreover, to define values or property names after creating the model, each property can be called individually and its value can be overwrited as:
```python
nwsas.eto = "ETo_"
nwsas.pumping_hours_per_day = 10
```


In [6]:
nwsas.print_properties()

Properties names:
    - Reference evapotranspiration (.eto): ETo_
    - Latitude (.lat): lat
    - Elevation (.elevation): elevation
    - Wind speed (.wind): wind
    - Solar radiation (.srad): srad
    - Min temperature (.tmin): tmin
    - Max temperature (.tmax): tmax
    - Avegarage temperature (.tavg): tavg


### 1.2. Setting required model parameters
To compute the irrigation water requierements, the share of cropland needs to be defined for each data point. That is, to specify the share each croptype has within each data point. To achieve this, first create a dictionary containing all the croplands of the region and assign a share for each. This share should be the default value that most of the data points should have. Specific values for different regions can also be defined, as explined later:
```python
crop_dic = {'crop1':0.5,'crop2':0.5,'crop3':0,...}
```
Then, use the `.set_cropland_share()` method to pass this dictionary to the model like:
```python
nwsas.set_cropland_share(crop_dic, inplace = True)
```
The option `inplace = True` is used to tell the model to store the dictionary in it.
Moreover, to define different cropland share values from the default one, a new dictionary can be passed to specific provinces, cities or regions, by passing a `geo_boundary` and a `boundary_name` for the region in question:
```python
nwsas.set_cropland_share({'crop1':0.7,'crop2':0.3,'crop3':0,...}, 
                          geo_boundary = 'province', 
                          boundary_name = 'province name', inplace = True)
```
The `geo_boundary` value needs to match an existent variable in the input dataframe and the `boundary_name` value should exist within the `geo_boundary` column.

In [7]:
crop_dic = {'dates':0.5,'vegetable':0.5,'olives':0}
nwsas.set_cropland_share(crop_dic, inplace = True)

nwsas.set_cropland_share({'dates':0.7,'vegetable':0.3,'olives':0}, 
                          geo_boundary = 'province', 
                          boundary_name = 'Jufrah', inplace = True)
nwsas.set_cropland_share({'dates':0,'vegetable':0.3,'olives':0.7}, 
                          geo_boundary = 'province', 
                          boundary_name = 'Gharyan', inplace = True)

### 1.3. Setting the ky and kc values
The yield responese factor (*ky*), is a coefficient that relates the water uses by a crop throughout the different growing seassons. A definition by the [FAO Irrigation and Drainage Paper](http://www.fao.org/3/i2800e/i2800e.pdf) is a follows:
>The yield response factor (Ky) captures the essence of the complex linkages
between production and water use by a crop, where many biological,
physical and chemical processes are involved. 

The Ky values are crop specific as:

>**Ky > 1**: crop response is very sensitive to water deficit with proportional larger yield reductions
when water use is reduced because of stress.  
**Ky < 1**: crop is more tolerant to water deficit, and recovers partially from stress, exhibiting less than proportional reductions in yield with reduced water use.  
**Ky = 1**: yield reduction is directly proportional to reduced water use.

The crop coefficient (*kc*) is a factor that relates the water requirements of a cropland during a specific growing seasson. A definition by the [FAO Irrigation and drainage paper 56](http://www.fao.org/3/x0490e/x0490e0a.htm) goes as follows:

>The coefficient integrates differences in the soil evaporation and crop transpiration rate between the crop and the grass reference surface. As soil evaporation may fluctuate daily as a result of rainfall or irrigation, the single crop coefficient expresses only the time-averaged (multi-day) effects of crop evapotranspiration.

To define the *ky* values, a dictionary containing the values for each crop type evaluated in the region needs to be passed to the `.ky_dict` parameter of the model. Similarly the *kc* values are passed to the `.kc_dict` parameter as a dictionary containing a list of values for each croptype (i.e. one for each season in order, i.e initial, development, mid and late season).

In [8]:
nwsas.ky_dict = {'dates':0.5,'vegetable':1.1,'olives':0.8}
nwsas.kc_dict = {'dates': [0.8,0.9,1,0.8], 
                 'vegetable':[0.5,1,1,0.8], 
                 'olives':[0.45,0.55,0.55,0.6]}

### 1.4. Calculating the reference evapotranspiration

In [9]:
nwsas.get_eto(inplace = True)

### 1.5. Calculating the effective rainfall

In [10]:
nwsas.get_effective_rainfall(inplace = True)

### 1.6. Calculating the kc values and standard evapotranspiration

In [11]:
nwsas.get_calendar_days(inplace = True)
nwsas.get_kc_values(inplace = True)

### 1.7. Geting the irrigation water demand

In [12]:
nwsas.get_water_demand(inplace = True)

## 2. Displaying and saving the results

In [13]:
nwsas.print_summary(geo_boundary = 'province')

province,Adrar,Biskra,Djelfa,El Oued,Gabes,Ghadamis,Ghardaia,Gharyan,Illizi,Jufrah,Kebili,Khenchela,Laghouat,Musrata,Ouargla,Tamanrasset,Tataouine,Tebessa,Tozeur
Irrigated area (ha),24101.816848,5749.995666,3449.972096,68999.985344,2874.983928,5353.213079,34499.998422,8557.616421,1770.396215,7963.713904,27412.511949,1149.970213,5749.994721,21082.919486,32699.346634,4868.857546,1833.775565,4599.970966,9617.446033
Water intensity (m3/ha),10934.163102,6946.421798,7133.087402,7940.932714,6974.411600,7279.941702,8844.340440,7489.888117,9809.723887,10463.012448,7758.884332,7498.094407,6981.596567,6155.054675,8570.160759,10839.006764,7072.361917,7565.076505,7472.868053
Water demand (Mm3),263.533196,39.941895,24.608952,547.924241,20.051321,38.971079,305.129731,64.095590,17.367098,83.324438,212.690509,8.622585,40.144143,129.766522,280.238657,52.773580,12.969124,34.799132,71.869905


In [14]:
output_file = r'nwsas_results/water_demand.csv'
nwsas.df.to_csv(output_file, index = False)